In [1]:
class Node:
    """A node in the suffix tree."""
    
    def __init__(self, start=None, end=None):
        self.children = {}  # Dictionary mapping characters to child nodes
        self.suffix_index = -1  # Index of suffix ending at this leaf (-1 for internal nodes)
        self.start = start  # Start index of the substring from original text
        self.end = end  # End index of the substring from original text
    
    def __str__(self):
        return f"Node(start={self.start}, end={self.end}, suffix_index={self.suffix_index})"


class SuffixTree:
    """Implementation of a naive suffix tree construction algorithm."""
    
    def __init__(self, text):
        """
        Create a suffix tree for the given text using the naive algorithm.
        
        Args:
            text (str): The text for which to create the suffix tree.
        """
        self.text = text + "$"  # Add terminal character
        self.root = Node()
        self._build_naive()
    
    def _edge_label(self, node):
        """Get the edge label leading to this node."""
        if node == self.root:
            return ""
        return self.text[node.start:node.end]
    
    def _build_naive(self):
        """
        Build the suffix tree using the naive algorithm described:
        - Start with first suffix
        - For each subsequent suffix i+1:
          - Find longest match from root
          - Split an edge if needed
          - Add a new leaf for the unmatched part
        """
        n = len(self.text)
        
        # Add the first suffix (entire string)
        first_leaf = Node(0, n)
        first_leaf.suffix_index = 0
        self.root.children[self.text[0]] = first_leaf
        
        # Add remaining suffixes one by one
        for i in range(1, n-1):  # n-1 because we added $ at the end
            current = self.root
            j = i  # Current position in the suffix being inserted
            
            # Find the longest path from root that matches a prefix of the current suffix
            while j < n:
                current_char = self.text[j]
                
                # If there's no path for the current character, we're done searching
                if current_char not in current.children:
                    break
                
                child = current.children[current_char]
                edge_label = self._edge_label(child)
                edge_len = len(edge_label)
                
                # Check how much of the edge matches the remaining suffix
                match_len = 0
                while match_len < edge_len and j + match_len < n and \
                      self.text[j + match_len] == edge_label[match_len]:
                    match_len += 1
                
                if match_len == edge_len:
                    # The entire edge matched, move down to the child node
                    j += edge_len
                    current = child
                else:
                    # Partial match, need to split the edge
                    break
            
            # At this point:
            # - current is the deepest node reached
            # - j is the position in the suffix where the match ended
            
            if j < n:
                if self.text[j] in current.children:
                    # Need to split an edge
                    child = current.children[self.text[j]]
                    edge_label = self._edge_label(child)
                    match_len = 0
                    
                    # Find the mismatch point
                    while j + match_len < n and match_len < len(edge_label) and \
                          self.text[j + match_len] == edge_label[match_len]:
                        match_len += 1
                    
                    # Create a new internal node where the split occurs
                    split_node = Node(child.start, child.start + match_len)
                    current.children[self.text[j]] = split_node
                    
                    # Adjust the original child node
                    child.start += match_len
                    split_node.children[self.text[child.start]] = child
                    
                    # Create a new leaf for the remaining suffix
                    new_leaf = Node(j + match_len, n)
                    new_leaf.suffix_index = i
                    split_node.children[self.text[j + match_len]] = new_leaf
                else:
                    # No split needed, just add a new leaf
                    new_leaf = Node(j, n)
                    new_leaf.suffix_index = i
                    current.children[self.text[j]] = new_leaf
    
    def visualize(self, node=None, depth=0, path=""):
        """
        Generate a string representation of the suffix tree.
        
        Args:
            node (Node, optional): Current node. Defaults to root.
            depth (int, optional): Current depth in tree. Defaults to 0.
            path (str, optional): Path from root to current node. Defaults to "".
            
        Returns:
            str: String representation of the tree.
        """
        if node is None:
            node = self.root
            result = "Suffix Tree:\n"
        else:
            # Show the edge label and suffix index if it's a leaf
            edge = self._edge_label(node)
            suffix_str = f" [suffix: {node.suffix_index}]" if node.suffix_index != -1 else ""
            result = "  " * depth + f"└── {edge}{suffix_str}\n"
            path += edge
        
        # Sort children by their first character for consistent visualization
        sorted_children = sorted(node.children.items(), key=lambda x: x[0])
        
        for char, child in sorted_children:
            result += self.visualize(child, depth + 1, path)
            
        return result
    
    def search(self, pattern):
        """
        Search for a pattern in the text.
        
        Args:
            pattern (str): The pattern to search for.
            
        Returns:
            list: List of starting indices where the pattern occurs in the text.
        """
        # Start at the root
        current = self.root
        
        # Keep track of how much of the pattern we've matched
        matched = 0
        
        while matched < len(pattern):
            # Get next character to match
            char = pattern[matched]
            
            # If there's no path for this character, pattern doesn't exist
            if char not in current.children:
                return []
            
            # Get the child node and its edge label
            child = current.children[char]
            edge_label = self._edge_label(child)
            
            # Check if the edge label matches part of the remaining pattern
            edge_len = len(edge_label)
            pattern_remaining = pattern[matched:]
            
            # If edge is longer than remaining pattern, check if pattern is a prefix of edge
            if edge_len > len(pattern_remaining):
                if pattern_remaining == edge_label[:len(pattern_remaining)]:
                    # Pattern is a prefix of this edge
                    break
                else:
                    # Mismatch
                    return []
            
            # Check if the edge matches the corresponding part of the pattern
            if edge_label != pattern_remaining[:edge_len]:
                return []
            
            # Move down to the child node
            current = child
            matched += edge_len
        
        # If we get here, the pattern exists in the tree
        # Now find all leaf nodes in the subtree to get all occurrences
        results = []
        self._collect_leaves(current, results)
        return sorted(results)
    
    def _collect_leaves(self, node, results):
        """
        Recursively collect all suffix indices in the subtree rooted at node.
        
        Args:
            node (Node): Current node.
            results (list): List to collect suffix indices.
        """
        if node.suffix_index != -1:
            results.append(node.suffix_index)
        
        for child in node.children.values():
            self._collect_leaves(child, results)




In [3]:
# Example usage
text = "CAGTCAGG"
tree1 = SuffixTree(text)

print(f"Original text: {text}")
print(tree1.visualize())

# Search examples
patterns = ["TC", "CAG", "GGT", "GTCA", "CAGTCAGG"]
# for pattern in patterns:
#     occurrences = tree1.search(pattern)
#     if occurrences:
#         print(f"Pattern '{pattern}' found at positions: {occurrences}")
#     else:
#         print(f"Pattern '{pattern}' not found")

Original text: CAGTCAGG
Suffix Tree:
  └── AG
    └── G$ [suffix: 5]
    └── TCAGG$ [suffix: 1]
  └── CAG
    └── G$ [suffix: 4]
    └── TCAGG$ [suffix: 0]
  └── G
    └── $ [suffix: 7]
    └── G$ [suffix: 6]
    └── TCAGG$ [suffix: 2]
  └── TCAGG$ [suffix: 3]

